In [1]:
%load_ext autoreload
%autoreload 2


import sys
import warnings


warnings.filterwarnings("ignore")
sys.path.append("../")

In [2]:
from modules.data.conll2003.prc import conll2003_preprocess

In [12]:
data_dir = "/home/eartemov/ae/work/conll2003/"

In [13]:
conll2003_preprocess(data_dir)

## IO markup

### Train

In [3]:
from modules.data import bert_data

In [4]:
data = bert_data.LearnData.create(
    train_df_path="/home/eartemov/ae/work/conll2003/eng.train.train.csv",
    valid_df_path="/home/eartemov/ae/work/conll2003/eng.testa.dev.csv",
    idx2labels_path="/home/eartemov/ae/work/conll2003/idx2labels2.txt",
    clear_cache=True,
    model_name="bert-base-cased"
)

The pre-trained model you are loading is a cased model but you have not set `do_lower_case` to False. We are setting `do_lower_case=False` for you but you may want to check this behavior.


In [5]:
from modules.models.bert_models import BERTBiLSTMAttnNCRF

In [6]:
model = BERTBiLSTMAttnNCRF.create(
    len(data.train_ds.idx2label), model_name="bert-base-cased",
    lstm_dropout=0., crf_dropout=0.3, nbest=len(data.train_ds.idx2label)-1)

build CRF...


In [7]:
from modules.train.train import NerLearner

In [8]:
num_epochs = 100

In [9]:
learner = NerLearner(
    model, data, "/home/eartemov/ae/work/models/conll2003-BERTBiLSTMAttnNCRF-base-IO.cpt",
    t_total=num_epochs * len(data.train_dl))

In [10]:
model.get_n_trainable_params()

2630289

In [ ]:
learner.fit(epochs=num_epochs)

### Eval

#### Load model

In [12]:
from modules.data import bert_data

In [13]:
data = bert_data.LearnData.create(
    train_df_path="/home/eartemov/ae/work/conll2003/eng.train.train.csv",
    valid_df_path="/home/eartemov/ae/work/conll2003/eng.testa.dev.csv",
    idx2labels_path="/home/eartemov/ae/work/conll2003/idx2labels2.txt",
    clear_cache=False,
    model_name="bert-base-cased"
)

INFO:pytorch_pretrained_bert.tokenization:loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased-vocab.txt from cache at /home/eartemov/.pytorch_pretrained_bert/5e8a2b4893d13790ed4150ca1906be5f7a03d6c4ddf62296c383f6db42814db2.e13dbb970cb325137104fb2e5f36fe865f27746c6b526f6352861b1980eb80b1


In [14]:
from modules.models.bert_models import BERTBiLSTMAttnNCRF

In [15]:
model = BERTBiLSTMAttnNCRF.create(
    len(data.train_ds.idx2label), model_name="bert-base-cased",
    lstm_dropout=0., crf_dropout=0.3, nbest=len(data.train_ds.idx2label)-1)

INFO:pytorch_pretrained_bert.modeling:loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased.tar.gz from cache at /home/eartemov/.pytorch_pretrained_bert/a803ce83ca27fecf74c355673c434e51c265fb8a3e0e57ac62a80e38ba98d384.681017f415dfb33ec8d0e04fe51a619f3f01532ecea04edbfd48c5d160550d9c
INFO:pytorch_pretrained_bert.modeling:extracting archive file /home/eartemov/.pytorch_pretrained_bert/a803ce83ca27fecf74c355673c434e51c265fb8a3e0e57ac62a80e38ba98d384.681017f415dfb33ec8d0e04fe51a619f3f01532ecea04edbfd48c5d160550d9c to temp dir /tmp/tmplvm3oqv1
INFO:pytorch_pretrained_bert.modeling:Model config {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "type_vocab_size": 2,
  "vocab_size": 28996
}



build CRF...


In [17]:
from modules.train.train import NerLearner

In [18]:
learner = NerLearner(
    model, data, "/home/eartemov/ae/work/models/conll2003-BERTBiLSTMAttnNCRF-base-IO.cpt")

In [19]:
model.get_n_trainable_params()

2630289

In [20]:
learner.load_model()

### Predict

In [21]:
from modules.data.bert_data import get_data_loader_for_predict

In [22]:
dl = get_data_loader_for_predict(data, df_path=data.valid_ds.config["df_path"])

In [23]:
preds = learner.predict(dl)

In [24]:
from sklearn_crfsuite.metrics import flat_classification_report

In [25]:
from modules.analyze_utils.utils import bert_labels2tokens, voting_choicer
from modules.analyze_utils.plot_metrics import get_bert_span_report

In [31]:
pred_tokens, pred_labels = bert_labels2tokens(dl, preds)
true_tokens, true_labels = bert_labels2tokens(dl, [x.bert_labels for x in dl.dataset])

In [32]:
assert pred_tokens == true_tokens
tokens_report = flat_classification_report(true_labels, pred_labels, labels=data.train_ds.idx2label[4:], digits=4)

In [33]:
print(tokens_report)

              precision    recall  f1-score   support

       I_ORG     0.9552    0.9461    0.9506      2005
         I_O     0.9966    0.9973    0.9970     41651
      I_MISC     0.9343    0.8956    0.9146      1255
       I_PER     0.9846    0.9863    0.9854      2848
       I_LOC     0.9727    0.9636    0.9681      1922

   micro avg     0.9918    0.9907    0.9913     49681
   macro avg     0.9687    0.9578    0.9631     49681
weighted avg     0.9917    0.9907    0.9912     49681



In [34]:
from modules.analyze_utils.plot_metrics import get_bert_span_report
from modules.analyze_utils.utils import voting_choicer, tokens2spans

In [35]:
print(get_bert_span_report(dl, preds, fn=voting_choicer))

              precision    recall  f1-score   support

         LOC     0.9227    0.9117    0.9172      1664
        MISC     0.9018    0.8639    0.8825       904
           O     0.9904    0.9920    0.9912     41631
         ORG     0.8654    0.8559    0.8606      1277
         PER     0.9460    0.9477    0.9468      1682

    accuracy                         0.9815     47158
   macro avg     0.9253    0.9142    0.9197     47158
weighted avg     0.9813    0.9815    0.9814     47158

